# Process lightning data from WWLN
This iPython notebook extracts lighning data from raw WWLN data files

Code by: Jasa Calogovic (Faculty of Geodesy, University of Zagreb)

Email: jcalogovic@geof.hr

In [1]:
# Load required packages
import numpy as np
import datetime as dt
from datetime import timedelta
import pandas as pd
import os
from bokeh.plotting import Figure, show, output_notebook, vplot
from bokeh.charts import Bar
from bokeh.io import gridplot
from bokeh.models import HoverTool, Span, Range1d, LinearAxis
from bokeh.models.sources import ColumnDataSource

from stormstats.storm import read_WWLN

output_notebook()

SyntaxError: only named arguments may follow *expression (storm.py, line 43)

## Various functions

In [ ]:
WWLN.Gen_date?

## Define parameters

In [ ]:
#data_path='../data/WWLN/Jan2016/'
data_path = "WWLN"

#start_time=
#end_time=
time_step=60   # in minutes
west_long_limit=0
east_long_limit=30
north_lat_limit=60
south_lat_limit=30
residuals_limit_us=2
minimum_no_stations=5

# Load data

In [ ]:
def read_WWLN(file):
    """Read WWLN file"""
    # Read the raw data and add headings
    tmp = pd.read_csv(file, parse_dates=True, header=None, names=['date','time','lat','lon','err','#sta'])
    # Generate a list of datetime objects with time to miliseconds
    list_dts = []
    for dvals, tvals, in zip(tmp['date'], tmp['time']):
        list_dts.append(gen_datetime(dvals, tvals))
    dtdf = pd.DataFrame(list_dts, columns=['datetime']) # Make a dt dataframe to join
    result = pd.concat([dtdf, tmp], axis=1)
    result = result.drop(['date','time'],axis=1)
    return result

In [ ]:
def gen_datetime(dvals, tvals):
    dvals = [int(t) for t in dvals.split('/')]    
    hh, mm, sec_micro = tvals.split(':')
    hh = int(hh)
    mm = int(mm)
    ss, mss = sec_micro.split('.')
    ss = int(ss)
    mss = int(mss)
    return dt.datetime(*dvals, hh, mm, ss, mss)

In [ ]:
#included_extenstions = ['loc']

#listfiles = [fn for fn in os.listdir(data_path) if any(fn.endswith(ext) for ext in included_extenstions)]

In [ ]:
#dates_data = []
#for file in listfiles:
#    dates_data.append(WWLN.Gen_date(file))

In [3]:
test = read_WWLN(file="WWLN/test.loc")
test.head()

,datetime,lat,lon,err,#sta
0,2016-01-01 00:00:00.258574,-7.6253,-64.6813,13.1,6
1,2016-01-01 00:00:00.205713,-5.1888,23.3846,4.1,5
2,2016-01-01 00:00:00.297611,-7.4574,-64.7405,16.9,6
3,2016-01-01 00:00:00.996208,-6.9209,-60.5694,6.2,13
4,2016-01-01 00:00:00.996221,-6.8842,-60.5744,21.0,6
